# Understanding CV
## Code adapted from 
- https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial12/Autoregressive_Image_Modeling.ipynb 
- https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial15/Vision_Transformer.ipynb

### Boilerplate

In [3]:
## Standard libraries
import os
import numpy as np 
import math
import random
from PIL import Image
from types import SimpleNamespace

## Imports for plotting
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgb
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
## PyTorch Lightning
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
import pytorch_lightning as pl
## Torchvision
import torchvision
from torchvision.datasets import MNIST
from torchvision import transforms

/var/folders/2v/0s_wd0cd04g3xtcdpppg90980000gn/T/ipykernel_15739/2311704251.py:14: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


In [ ]:
device = torch.device("mps") if torch.mps.is_available() else torch.device("cpu")
print("Using device", device)
DATASET_PATH = "../data"
CHECKPOINT_PATH = "./saved_models"
pl.seed_everything(42)